<a href="https://colab.research.google.com/github/ryu622/gnn-counterattack-xai-v2/blob/fix%2Ffile-clean/GAT_CounterAttack_Prediction_Train_Scientific8_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#シード値
import random
import os
import numpy as np
import torch

def set_seed(seed=42):
    # Python自体の乱数固定
    random.seed(seed)
    # OS環境の乱数固定
    os.environ['PYTHONHASHSEED'] = str(seed)
    # Numpyの乱数固定
    np.random.seed(seed)
    # PyTorchの乱数固定
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) # マルチGPUの場合
    # 計算の決定論的挙動を強制（これを入れると少し遅くなることがありますが、再現性は完璧になります）
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# 好きな数字（42が一般的）で固定
set_seed(42)

In [ ]:
#GoogleDriveをマウント

from google.colab import drive

# Google Driveを仮想ファイルシステムにマウント
drive.mount('/content/drive')

In [ ]:
# 必須モジュールのインポート
!pip install torch_geometric

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from sklearn.metrics import classification_report
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch import optim

import pandas as pd
import os
from glob import glob
import numpy as np
import re
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

# 表示設定
np.set_printoptions(suppress=True, precision=3)
pd.set_option('display.precision', 3)    # 小数点以下の表示桁
pd.set_option('display.max_rows', 50)   # 表示する行数の上限
pd.set_option('display.max_columns', 15)  # 表示する列数の上限
%precision 3

使用データは以前までと同様

In [ ]:
import torch
from torch_geometric.loader import DataLoader

# ==========================================
# ロード・クリーンアップ・最終確認
# ==========================================
v7_load_path = "/content/drive/MyDrive/GNN_Football_Analysis/Processed_Data/gnn_data_v14_final.pt"

try:
    print(f"v7 最終データをロード中: {v7_load_path}")
    # 統合済みファイルをロード
    checkpoint = torch.load(v7_load_path, weights_only=False)

    # v7 の Builder ですでに 7次元特徴量 (x, y, vx, vy, dist_goal, dist_ball, team_id)
    # を付与しているため、基本的にはそのまま DataLoader に渡せます。
    train_set = checkpoint['train_data']
    test_set = checkpoint['test_data']

    # DataLoader を構築 (バッチサイズはメモリに合わせて調整してください)
    train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_set, batch_size=32, shuffle=False)

    print(f"--- ロード完了 ---")
    print(f"訓練セット: {len(train_set)} 枚")
    print(f"テストセット: {len(test_set)} 枚")

    # 【最重要チェック】ノード数と次元数の確認
    sample_train = train_set[0]
    sample_test = test_set[0]

    # 期待値: [23, 7] (22人 + ボール1つ、7種類の特徴量)
    print(f"訓練データの形状: {sample_train.x.shape}")
    print(f"テストデータの形状: {sample_test.x.shape}")

    # 1. 次元数チェック
    if sample_train.x.shape[1] == 7:
        print("次元数: OK (7次元)")
    else:
        print(f"次元数警告: {sample_train.x.shape[1]}次元になっています。")

    # 2. ノード数チェック
    if sample_train.x.shape[0] == 23:
        print("ノード数: OK (23ノード固定)")
    else:
        print(f"ノード数警告: {sample_train.x.shape[0]}ノードになっています。")

    # 3. 速度データの存在チェック
    # 2列目(vx)の絶対値平均が0でなければ、速度が正しく入っています
    v_mean = torch.abs(sample_train.x[:, 2]).mean().item()
    if v_mean > 0.01:
        print(f"物理量チェック: OK (平均速度属性を確認)")
    else:
        print("物理量警告: 速度が0に張り付いています。Builderを再確認してください。")

    if sample_train.x.shape[1] == 7 and v_mean > 0.01:
        print("\nすべての準備が整いました。PIGNN 学習を開始してください。")

except FileNotFoundError:
    print(f"ファイルが見つかりません。パスを確認してください: {v7_load_path}")
except Exception as e:
    print(f"エラー発生: {e}")

モデルの定義（PIGNNのオリジナルクラス）

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing, global_mean_pool
from torch_geometric.utils import softmax
from torch_geometric.data import Data
import os

# ==========================================
# 1. 前処理関数の定義
# ==========================================
def preprocess_batch(data, device):
    # スケーリングはBuilder側で行われているため、ここでは型変換とDevice転送に集中
    data.x = data.x.float()
    data.pos = data.pos.float()
    data.vel = data.vel.float()
    return data.to(device)

# ==========================================
# 2. モデル定義（チーム属性によるメッセージ分岐の実装）
# ==========================================
class PIGNNLayer(MessagePassing):
    def __init__(self, in_channels, out_channels, tau=1.5):
        super(PIGNNLayer, self).__init__(aggr='add')
        self.tau = tau
        self.lin = nn.Linear(in_channels, out_channels)
        self.att = nn.Parameter(torch.Tensor(1, out_channels * 2))
        nn.init.xavier_uniform_(self.att)

    def forward(self, x, edge_index, pos, vel):
        h = self.lin(x)
        # チームID(index 6)をメッセージパッシングに渡す
        return self.propagate(edge_index, x=h, pos=pos, vel=vel, team=x[:, 6:7])

    def message(self, x_i, x_j, pos_i, pos_j, vel_i, vel_j, edge_index_i, team_i, team_j):
        # 理論1: 未来位置予測ベースのバイアス
        pos_i_pred = pos_i + vel_i * self.tau
        pos_j_pred = pos_j + vel_j * self.tau
        dist_future = torch.norm(pos_i_pred - pos_j_pred, dim=-1, keepdim=True)
        physics_bias = torch.exp(-dist_future / 2.0)

        # 【追加】チーム関係分岐: 味方なら+0.5, 敵なら-0.5の注目度補正
        is_teammate = (team_i == team_j).float()
        team_bias = torch.where(is_teammate > 0.5, 0.5, -0.5)

        alpha = torch.cat([x_i, x_j], dim=-1)
        alpha = (alpha * self.att).sum(dim=-1, keepdim=True)
        # 物理バイアス + チーム属性バイアス を統合
        alpha = softmax(F.leaky_relu(alpha) + physics_bias + team_bias, edge_index_i)

        return alpha * x_j

class PIGNNClassifier(nn.Module):
    def __init__(self, hidden_channels=64):
        super(PIGNNClassifier, self).__init__()
        self.conv1 = PIGNNLayer(7, hidden_channels)
        self.conv2 = PIGNNLayer(hidden_channels, hidden_channels)
        self.lin = nn.Linear(hidden_channels, 2)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        pos, vel = data.pos, data.vel

        x = F.elu(self.conv1(x, edge_index, pos, vel))
        x = self.conv2(x, edge_index, pos, vel)

        x = global_mean_pool(x, batch)
        return F.log_softmax(self.lin(x), dim=1)

# ==========================================
# 3. 理論修正：集団運動学的制約（L_phys）
# ==========================================
def pignn_theoretical_loss(output, target, data, alpha=0.1):
    # L_task: クラス重み付き（Success:3.3倍）
    weights = torch.tensor([1.0, 3.3], device=output.device)
    loss_task = F.nll_loss(output, target, weight=weights)

    # L_phys: 集団推進力制約
    # Success確率が高いほど、チーム全体の重心(全ノード平均)が右(+vx)であることを求める
    probs = torch.exp(output)[:, 1]

    # グラフごとの平均vxを算出（選手全員の動きを統合）
    # data.batch を用いて各グラフ(シーン)の平均vxを計算
    batch_size = output.size(0)
    # 各グラフの平均vxを計算
    avg_vxs = []
    for i in range(batch_size):
        mask = (data.batch == i)
        avg_vxs.append(torch.mean(data.vel[mask, 0])) # 各シーンの全ノード平均vx

    avg_vxs = torch.stack(avg_vxs)

    # 物理損失：Success確率 × ReLU(-平均vx)
    # シーン全体が左に流れているのに「成功」と出すと強く罰せられる
    loss_phys = torch.mean(probs * torch.relu(-avg_vxs))

    total_loss = loss_task + (alpha * loss_phys)
    return total_loss, loss_task, loss_phys

# ==========================================
# 4. 学習・評価ループ
# ==========================================
def train_pignn_epoch_dynamic(model, loader, optimizer, device, epoch):
    model.train()
    total_loss, total_phys = 0, 0

    # アニーリング（後半10エポック以降で物理を強化）
    if epoch <= 10:
        current_alpha = 0.1
    else:
        current_alpha = min(0.1 + (epoch - 10) * 0.2, 5.0)

    for data in loader:
        data = preprocess_batch(data, device)
        optimizer.zero_grad()

        out = model(data)
        loss, _, l_phys = pignn_theoretical_loss(out, data.y.view(-1), data, alpha=current_alpha)

        loss.backward()
        optimizer.step()

        total_loss += loss.item() * data.num_graphs
        total_phys += l_phys.item() * data.num_graphs

    return total_loss / len(loader.dataset), total_phys / len(loader.dataset), current_alpha

def test_pignn(model, loader, device):
    model.eval()
    correct = 0
    with torch.no_grad():
        for data in loader:
            data = preprocess_batch(data, device)
            out = model(data)
            pred = out.argmax(dim=1)
            correct += (pred == data.y.view(-1)).sum().item()
    return correct / len(loader.dataset)

In [ ]:
#実行

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ハイパーパラメータ（あなたの設定を維持）
EPOCHS = 50
LR = 0.0005

history = {
    'total_loss': [],
    'physics_loss': [],
    'test_acc': [],
    'alpha': []
}

# モデルと最適化手法の初期化
model = PIGNNClassifier(hidden_channels=64).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

# ファイル名
save_path = 'best_pignn_theoretical_V1.pth'

print(f"PIGNN学習開始 (Device: {device} | 物理・分類 理論統合モード)")
print(f"Input Features: 7 [x, y, vx, vy, (1-px), dist_ball, team_id]")

for epoch in range(1, EPOCHS + 1):
    # 先ほど修正した、勾配が繋がった train_pignn_epoch_dynamic を呼び出し
    avg_loss, avg_phys, current_alpha = train_pignn_epoch_dynamic(model, train_loader, optimizer, device, epoch)

    # テスト評価
    acc = test_pignn(model, test_loader, device)

    # 履歴の保存
    history['total_loss'].append(avg_loss)
    history['physics_loss'].append(avg_phys)
    history['test_acc'].append(acc)
    history['alpha'].append(current_alpha)

    # 進捗表示：Phys_L が 0.7439 から変化しているか注目してください
    if epoch % 5 == 0 or epoch == 1:
        # 理論に基づいた Phys_L の挙動を確認しやすく表示
        print(f"Epoch {epoch:03d} | Alpha: {current_alpha:.2f} | Loss: {avg_loss:.4f} | Phys_L (Penalty): {avg_phys:.6f} | Acc: {acc:.4f}")

    # 保存ロジック：精度が向上したときのみ保存
    if epoch == 1 or acc > max(history['test_acc'][:-1]):
        torch.save(model.state_dict(), save_path)
        print(f" >> [Update] Model Saved: {save_path} (Best Acc: {acc:.4f})")

print(f"\n学習完了。最高精度: {max(history['test_acc']):.4f}")

In [ ]:
# ==========================================
# 5. 最終評価と物理的妥当性レポート (集団運動学対応版)
# ==========================================
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

# 1. 保存したモデルをロード
model_path = 'best_pignn_theoretical_V1.pth'
model.load_state_dict(torch.load(model_path))
model.eval()

all_preds = []
all_labels = []
success_team_vxs = [] # 成功と予測した時の「チーム平均速度」を記録

# 2. テストデータで最終予測
with torch.no_grad():
    for data in test_loader:
        data = preprocess_batch(data, device)

        out = model(data)
        pred = out.argmax(dim=1)

        # --- 物理的妥当性のためのデータ抽出 ---
        # 各グラフ(シーン)ごとのチーム平均vxを計算
        for i in range(out.size(0)):
            mask = (data.batch == i)
            avg_vx = torch.mean(data.vel[mask, 0]).item() # シーン内の全ノード平均vx

            # AIが「Success(1)」と予測したシーンの平均vxだけをリストに溜める
            if pred[i] == 1:
                success_team_vxs.append(avg_vx)

        all_preds.extend(pred.cpu().numpy())
        all_labels.extend(data.y.view(-1).cpu().numpy())

# 3. レポート表示
print("\n" + "="*60)
print("       PIGNN 最終評価結果 (物理理論・集団運動統合モデル)")
print("="*60)
print(classification_report(all_labels, all_preds, target_names=['Failure (0)', 'Success (1)']))

# 4. 物理的妥当性の検証結果 (卒論のメイン考察)
print("\n" + "="*60)
print("       物理的整合性 検証レポート (集団推進力ベース)")
print("="*60)
if len(success_team_vxs) > 0:
    avg_team_vx = np.mean(success_team_vxs)
    positive_ratio = np.sum(np.array(success_team_vxs) > 0) / len(success_team_vxs)

    print(f"成功予測シーンにおける『チーム平均速度』vx: {avg_team_vx:.4f} m/s")
    print(f"成功予測シーンにおけるチーム右向き(正)の割合: {positive_ratio*100:.1f} %")

    # 基準をデータの現実に合わせて調整 (60-70%以上なら物理的に機能しているとみなす)
    if positive_ratio > 0.65:
        print(">> 判定: 物理的整合性あり。モデルはチーム全体の『集団推進力』を根拠にしています。")
    else:
        print(">> 判定: 物理的矛盾あり。データの反転ミスか、学習のバイアスが強すぎます。")
else:
    print("Successと予測されたデータがありません。")

# 5. 混同行列の描画
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Fail', 'Success'], yticklabels=['Fail', 'Success'])
plt.title('Confusion Matrix (Team-Physics PIGNN)')
plt.ylabel('Actual (True)')
plt.xlabel('Predicted')
plt.show()

In [ ]:
import torch

def verify_pt_file(file_path):
    # ファイルをロード
    data_list = torch.load(file_path, weights_only=False)
    if isinstance(data_list, dict):
        # もし train_data, test_data に分かれて保存されている場合
        data_list = data_list.get('test_data', []) or data_list.get('train_data', [])

    success_right = 0
    success_total = 0

    for data in data_list:
        # ラベルがSuccess(1)のものだけを抽出
        if data.y.item() == 1:
            success_total += 1

            # ボールのvx (ノード特徴量 x の index 2) を取得
            # ノードの最後がボールという前提
            ball_vx = data.x[-1, 2].item()

            # 物理的に右(正)に動いているか判定
            if ball_vx > 0:
                success_right += 1

    if success_total == 0:
        print("Successラベルのデータが見つかりませんでした。")
        return

    ratio = (success_right / success_total) * 100
    print(f"--- .ptファイル内部検証結果 ---")
    print(f"Successラベルの総数: {success_total}")
    print(f"そのうち物理的に右(vx > 0)を向いている数: {success_right}")
    print(f"【結論】 成功シーンの物理的正解率: {ratio:.1f}%")

    if ratio < 50:
        print("\n警告: 前処理が失敗しています。")
        print("ラベルは『成功』なのに、座標上は『逆走』しているデータが過半数です。")
    elif ratio < 80:
        print("\n注意: 整合性が不十分です。バックパス等のノイズが多いか、反転ミスが混ざっています。")
    else:
        print("\n合格: 物理とラベルが一致しています。")

# 実行
verify_pt_file('/content/drive/MyDrive/GNN_Football_Analysis/Processed_Data/gnn_data_v14_final.pt')

In [ ]:
import torch

def verify_pt_players_velocity(file_path):
    # ファイルをロード
    data_list = torch.load(file_path, weights_only=False)
    if isinstance(data_list, dict):
        data_list = data_list.get('test_data', []) or data_list.get('train_data', [])

    success_total = 0
    group_right_count = 0  # 全選手の平均が右向き
    group_left_count = 0   # 全選手の平均が左向き

    print(f"--- 選手全員の速度ベクトルによる詳細検証 ---")

    for i, data in enumerate(data_list):
        if data.y.item() == 1:  # Successラベルのみ
            success_total += 1

            # data.x[:, 2] は全ノード（選手+ボール）の vx
            # ボール（最後）を除いた選手全員の平均 vx を計算
            player_vxs = data.x[:-1, 2]
            avg_player_vx = torch.mean(player_vxs).item()

            if avg_player_vx > 0:
                group_right_count += 1
            else:
                group_left_count += 1

    print(f"Successラベル総数: {success_total}")
    print(f"  └ 選手集団が『右』へ移動中: {group_right_count} シーン ({group_right_count/success_total*100:.1f}%)")
    print(f"  └ 選手集団が『左』へ移動中: {group_left_count} シーン ({group_left_count/success_total*100:.1f}%)")

    print("\n【客観的な診断】")
    if group_left_count > group_right_count:
        print("致命的エラー: 『成功シーン』の多くで、選手全員が左に向かって走っています。")
        print("   原因：flip_factor（反転処理）が真逆、または適用されていません。")
    else:
        print("選手集団の方向は概ね一致していますが、ボールの方向とズレがある可能性があります。")

verify_pt_players_velocity('/content/drive/MyDrive/GNN_Football_Analysis/Processed_Data/gnn_data_v14_final.pt')

In [ ]:
import torch
from torch_geometric.data import Data

def run_leakage_diagnostic(model, loader, device):
    model.eval()
    # 特徴量ラベル: 0:x, 1:y, 2:vx, 3:vy, 4:dist_goal, 5:dist_ball, 6:team_id
    feature_names = ["x", "y", "vx", "vy", "dist_goal", "dist_ball", "team_id"]

    print(f"{'Removed Feature':<15} | {'Test Acc':<10} | {'Recall (S)':<10}")
    print("-" * 45)

    # ベースライン（全特徴量あり）
    base_acc = test_pignn(model, loader, device)
    print(f"{'None (Baseline)':<15} | {base_acc:.4f}")

    with torch.no_grad():
        for i in range(7):
            correct = 0
            tp = 0 # True Positive
            fn = 0 # False Negative

            for data in loader:
                data = preprocess_batch(data, device)

                # 特定の特徴量をゼロに置き換える
                x_shuffled = data.x.clone()
                x_shuffled[:, i] = 0.0

                # 推論
                out = model(Data(x=x_shuffled, edge_index=data.edge_index,
                                 batch=data.batch, pos=data.pos, vel=data.vel))
                pred = out.argmax(dim=1)

                # 精度計算
                y_true = data.y.view(-1)
                correct += (pred == y_true).sum().item()

                # Recall (Success) 計算
                tp += ((pred == 1) & (y_true == 1)).sum().item()
                fn += ((pred == 0) & (y_true == 1)).sum().item()

            acc = correct / len(loader.dataset)
            recall = tp / (tp + fn) if (tp + fn) > 0 else 0
            print(f"{feature_names[i]:<15} | {acc:.4f}     | {recall:.4f}")

# 実行
run_leakage_diagnostic(model, test_loader, device)

ゴールへの距離は成功の定義が３５ｍ地点への侵入と定義してしまっているせいで、過度に依存している。そのため、この特徴量を抜いて修正してみる

修正

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing, global_mean_pool
from torch_geometric.utils import softmax
import numpy as np

# ==========================================
# 1. 前処理関数の定義（ここでリークを遮断！）
# ==========================================
def preprocess_batch(data, device):
    data.x = data.x.float()

    # 【核心】index 4 (dist_goal) を除外してリークを物理的に遮断
    # 元: [x, y, vx, vy, dist_goal, dist_ball, team_id] (7次元)
    # 新: [x, y, vx, vy, dist_ball, team_id] (6次元)
    x_no_leak = torch.cat([data.x[:, :4], data.x[:, 5:]], dim=1)
    data.x = x_no_leak

    data.pos = data.pos.float()
    data.vel = data.vel.float()
    return data.to(device)

# ==========================================
# 2. モデル定義（入力6次元・チーム属性分岐対応）
# ==========================================
class PIGNNLayer(MessagePassing):
    def __init__(self, in_channels, out_channels, tau=1.5):
        super(PIGNNLayer, self).__init__(aggr='add')
        self.tau = tau
        self.lin = nn.Linear(in_channels, out_channels)
        self.att = nn.Parameter(torch.Tensor(1, out_channels * 2))
        nn.init.xavier_uniform_(self.att)

    def forward(self, x, edge_index, pos, vel):
        h = self.lin(x)
        # 6次元化に伴い、team_id は index 5 に移動している
        return self.propagate(edge_index, x=h, pos=pos, vel=vel, team=x[:, 5:6])

    def message(self, x_i, x_j, pos_i, pos_j, vel_i, vel_j, edge_index_i, team_i, team_j):
        # 理論1: 未来位置予測ベースのバイアス
        pos_i_pred = pos_i + vel_i * self.tau
        pos_j_pred = pos_j + vel_j * self.tau
        dist_future = torch.norm(pos_i_pred - pos_j_pred, dim=-1, keepdim=True)
        physics_bias = torch.exp(-dist_future / 2.0)

        # チーム関係分岐: 味方なら注目度を上げ、敵なら抑制する
        is_teammate = (team_i == team_j).float()
        team_bias = torch.where(is_teammate > 0.5, 0.5, -0.5)

        alpha = torch.cat([x_i, x_j], dim=-1)
        alpha = (alpha * self.att).sum(dim=-1, keepdim=True)
        alpha = softmax(F.leaky_relu(alpha) + physics_bias + team_bias, edge_index_i)

        return alpha * x_j

class PIGNNClassifier(nn.Module):
    def __init__(self, hidden_channels=64):
        super(PIGNNClassifier, self).__init__()
        # 入力次元を 6 に変更
        self.conv1 = PIGNNLayer(6, hidden_channels)
        self.conv2 = PIGNNLayer(hidden_channels, hidden_channels)
        self.lin = nn.Linear(hidden_channels, 2)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        pos, vel = data.pos, data.vel

        x = F.elu(self.conv1(x, edge_index, pos, vel))
        x = self.conv2(x, edge_index, pos, vel)

        x = global_mean_pool(x, batch)
        return F.log_softmax(self.lin(x), dim=1)

# ==========================================
# 3. 損失関数の定義（集団運動学的制約）
# ==========================================
def pignn_theoretical_loss(output, target, data, alpha=0.1):
    weights = torch.tensor([1.0, 3.3], device=output.device)
    loss_task = F.nll_loss(output, target, weight=weights)

    probs = torch.exp(output)[:, 1]

    # グラフごとのチーム平均vxを算出
    batch_size = output.size(0)
    avg_vxs = []
    for i in range(batch_size):
        mask = (data.batch == i)
        avg_vxs.append(torch.mean(data.vel[mask, 0]))

    avg_vxs = torch.stack(avg_vxs)
    loss_phys = torch.mean(probs * torch.relu(-avg_vxs))

    total_loss = loss_task + (alpha * loss_phys)
    return total_loss, loss_task, loss_phys

# ==========================================
# 4. 学習・評価関数
# ==========================================
def train_pignn_epoch_dynamic(model, loader, optimizer, device, epoch):
    model.train()
    total_loss, total_phys = 0, 0
    if epoch <= 10:
        current_alpha = 0.1
    else:
        current_alpha = min(0.1 + (epoch - 10) * 0.2, 5.0)

    for data in loader:
        data = preprocess_batch(data, device)
        optimizer.zero_grad()
        out = model(data)
        loss, _, l_phys = pignn_theoretical_loss(out, data.y.view(-1), data, alpha=current_alpha)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
        total_phys += l_phys.item() * data.num_graphs
    return total_loss / len(loader.dataset), total_phys / len(loader.dataset), current_alpha

def test_pignn(model, loader, device):
    model.eval()
    correct = 0
    with torch.no_grad():
        for data in loader:
            data = preprocess_batch(data, device)
            out = model(data)
            pred = out.argmax(dim=1)
            correct += (pred == data.y.view(-1)).sum().item()
    return correct / len(loader.dataset)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ハイパーパラメータ
EPOCHS = 50
LR = 0.0005

history = {
    'total_loss': [],
    'physics_loss': [],
    'test_acc': [],
    'alpha': []
}

# モデルの初期化（入力6次元版）
model = PIGNNClassifier(hidden_channels=64).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

# ファイル名（リーク対策版として保存）
save_path = 'best_pignn_v12_no_leak.pth'

print(f"PIGNN学習開始 (Device: {device} | リーク対策・集団物理統合モード)")
# 修正：モデルに実際に入力されるのは6次元
print(f"Input Features (Processed): 6 [x, y, vx, vy, dist_ball, team_id]")
print(f"Skipped Feature: dist_goal (to prevent data leakage)")

for epoch in range(1, EPOCHS + 1):
    # 内部で preprocess_batch が走り、6次元に変換される
    avg_loss, avg_phys, current_alpha = train_pignn_epoch_dynamic(model, train_loader, optimizer, device, epoch)

    # テスト評価
    acc = test_pignn(model, test_loader, device)

    # 履歴の保存
    history['total_loss'].append(avg_loss)
    history['physics_loss'].append(avg_phys)
    history['test_acc'].append(acc)
    history['alpha'].append(current_alpha)

    # 進捗表示
    if epoch % 5 == 0 or epoch == 1:
        print(f"Epoch {epoch:03d} | Alpha: {current_alpha:.2f} | Loss: {avg_loss:.4f} | Phys_L: {avg_phys:.6f} | Acc: {acc:.4f}")

    # 最高精度の更新保存
    if epoch == 1 or acc > max(history['test_acc'][:-1]):
        torch.save(model.state_dict(), save_path)
        print(f" >> [Update] Model Saved: {save_path} (Best Acc: {acc:.4f})")

print(f"\n学習完了。最高精度: {max(history['test_acc']):.4f}")

In [ ]:
# ==========================================
# 5. 最終評価と物理的妥当性レポート (集団運動学対応版)
# ==========================================
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

# 1. 保存した【リーク対策版】モデルをロード
# 学習時に指定した save_path に合わせてください
model_path = 'best_pignn_v12_no_leak.pth'
model.load_state_dict(torch.load(model_path))
model.eval()

all_preds = []
all_labels = []
success_team_vxs = [] # 成功と予測した時の「チーム平均速度」を記録

# 2. テストデータで最終予測
with torch.no_grad():
    for data in test_loader:
        # 重要：評価時も preprocess_batch を通して 7次元→6次元 に変換する
        data = preprocess_batch(data, device)

        out = model(data)
        pred = out.argmax(dim=1)

        # --- 物理的妥当性のためのデータ抽出 ---
        for i in range(out.size(0)):
            mask = (data.batch == i)
            # data.vel は [全ノード, 2] なのでそのまま平均をとる
            avg_vx = torch.mean(data.vel[mask, 0]).item()

            if pred[i] == 1:
                success_team_vxs.append(avg_vx)

        all_preds.extend(pred.cpu().numpy())
        all_labels.extend(data.y.view(-1).cpu().numpy())

# 3. レポート表示
print("\n" + "="*60)
print("       PIGNN 最終評価結果 (リーク対策・集団物理統合モデル)")
print("="*60)
# ターゲットネームを実際のデータに合わせて表示
print(classification_report(all_labels, all_preds, target_names=['Failure (0)', 'Success (1)']))

# 4. 物理的妥当性の検証結果 (卒論のメイン考察)
print("\n" + "="*60)
print("       物理的整合性 検証レポート (集団推進力ベース)")
print("="*60)
if len(success_team_vxs) > 0:
    avg_team_vx = np.mean(success_team_vxs)
    positive_ratio = np.sum(np.array(success_team_vxs) > 0) / len(success_team_vxs)

    print(f"成功予測シーンにおける『チーム平均速度』vx: {avg_team_vx:.4f} m/s")
    print(f"成功予測シーンにおけるチーム右向き(正)の割合: {positive_ratio*100:.1f} %")

    # 判定基準：カオスなデータ(整合性60-70%)でも、50%を明確に超えれば物理制約の効果ありと言える
    if positive_ratio > 0.60:
        print(">> 判定: 物理的整合性あり。モデルは『集団の推進力』を根拠としています。")
    else:
        print(">> 判定: 物理的矛盾またはデータの限界。Alpha値を調整するか、データの精査が必要です。")
else:
    print("Successと予測されたデータがありません。")

# 5. 混同行列の描画
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Fail', 'Success'], yticklabels=['Fail', 'Success'])
plt.title('Confusion Matrix (v12 No-Leak PIGNN)')
plt.ylabel('Actual (True)')
plt.xlabel('Predicted')
plt.show()

以上がPIGNNのベースラインモデル

ただのMLP

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import global_mean_pool

# ==========================================
# 1. MLPモデルの定義
# ==========================================
class SimpleMLPClassifier(nn.Module):
    def __init__(self, in_channels=6, hidden_channels=64):
        super(SimpleMLPClassifier, self).__init__()
        # グラフ畳み込みを使わず、全結合層のみで構成
        self.mlp = nn.Sequential(
            nn.Linear(in_channels, hidden_channels),
            nn.ReLU(),
            nn.Linear(hidden_channels, hidden_channels),
            nn.ReLU(),
            nn.Linear(hidden_channels, 2)
        )

    def forward(self, data):
        # 1. グラフ構造を無視して、全ノードの平均をとる (Pooling)
        # [num_nodes, 6] -> [batch_size, 6]
        x = global_mean_pool(data.x, data.batch)

        # 2. 全結合層に投入
        return F.log_softmax(self.mlp(x), dim=1)

# ==========================================
# 2. MLP用の学習ループ (物理損失なしの標準的な学習)
# ==========================================
def train_mlp_baseline(model, loader, optimizer, device):
    model.train()
    total_loss = 0
    # MLPは比較用なので、重み付きクロスエントロピーのみで学習
    weights = torch.tensor([1.0, 3.3], device=device)

    for data in loader:
        data = preprocess_batch(data, device) # 6次元化
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out, data.y.view(-1), weight=weights)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
    return total_loss / len(loader.dataset)

# --- 実行コード ---
mlp_model = SimpleMLPClassifier(in_channels=6).to(device)
mlp_optimizer = torch.optim.Adam(mlp_model.parameters(), lr=0.0005)

print("MLP Baseline 学習開始...")
for epoch in range(1, 51):
    loss = train_mlp_baseline(mlp_model, train_loader, mlp_optimizer, device)
    if epoch % 10 == 0:
        acc = test_pignn(mlp_model, test_loader, device) # 評価は共通
        print(f"Epoch {epoch:03d} | Loss: {loss:.4f} | Acc: {acc:.4f}")

torch.save(mlp_model.state_dict(), 'mlp_baseline.pth')

In [ ]:
# MLPの最終評価
print("\n" + "="*60)
print("       MLP Baseline 最終評価レポート")
print("="*60)

mlp_model.eval()
all_preds_mlp = []
all_labels_mlp = []
success_vxs_mlp = []

with torch.no_grad():
    for data in test_loader:
        data = preprocess_batch(data, device) # 6次元化
        out = mlp_model(data)
        pred = out.argmax(dim=1)

        for i in range(out.size(0)):
            mask = (data.batch == i)
            avg_vx = torch.mean(data.vel[mask, 0]).item()
            if pred[i] == 1:
                success_vxs_mlp.append(avg_vx)

        all_preds_mlp.extend(pred.cpu().numpy())
        all_labels_mlp.extend(data.y.view(-1).cpu().numpy())

print(classification_report(all_labels_mlp, all_preds_mlp, target_names=['Fail', 'Success']))

if len(success_vxs_mlp) > 0:
    pos_ratio_mlp = np.sum(np.array(success_vxs_mlp) > 0) / len(success_vxs_mlp)
    print(f"MLPの成功予測時・右向き整合性: {pos_ratio_mlp*100:.1f} %")
else:
    print("Success予測なし")

敵味方で条件分岐させて学習させるモデル？

In [ ]:
class PIGNNLayer(MessagePassing):
    def __init__(self, in_channels, out_channels, tau=1.5):
        super(PIGNNLayer, self).__init__(aggr='add')
        self.tau = tau
        self.lin = nn.Linear(in_channels, out_channels)
        self.att = nn.Parameter(torch.Tensor(1, out_channels * 2))
        nn.init.xavier_uniform_(self.att)

    def forward(self, x, edge_index, pos, vel):
        h = self.lin(x)
        # team_id (index 6) を抽出してメッセージパッシングに渡す
        return self.propagate(edge_index, x=h, pos=pos, vel=vel, team=x[:, 6:7])

    def message(self, x_i, x_j, pos_i, pos_j, vel_i, vel_j, edge_index_i, team_i, team_j):
        # 1. 物理未来位置計算
        pos_i_pred = pos_i + vel_i * self.tau
        pos_j_pred = pos_j + vel_j * self.tau
        dist_future = torch.norm(pos_i_pred - pos_j_pred, dim=-1, keepdim=True)
        physics_bias = torch.exp(-dist_future / 2.0)

        # 2. 【核心】チーム属性による分岐
        # 味方なら 1.0, 敵なら 0.0
        is_teammate = (team_i == team_j).float()

        # 味方同士ならアテンションを強め、敵なら弱める（守備的干渉）
        # この +0.5 と -0.5 が戦術的な意味を生む
        team_bias = torch.where(is_teammate > 0.5, torch.tensor(0.5, device=is_teammate.device),
                                torch.tensor(-0.5, device=is_teammate.device))

        alpha = torch.cat([x_i, x_j], dim=-1)
        alpha = (alpha * self.att).sum(dim=-1, keepdim=True)
        alpha = softmax(F.leaky_relu(alpha) + physics_bias + team_bias, edge_index_i)

        return alpha * x_j

PFI

In [ ]:
import torch
import os

# 1. 保存した最新の「物理損失統合版」モデルパスを指定
# 学習時に保存した名前に正確に書き換えてください
save_path = "best_pignn_physics_integrated_v1.pth"

# 2. モデルのインスタンス化
model = PIGNNClassifier(hidden_channels=64)

# 3. 重みのロード
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if os.path.exists(save_path):
    # ロード（最新のPyTorchでは weights_only=True が推奨）
    checkpoint = torch.load(save_path, map_location=device, weights_only=True)
    model.load_state_dict(checkpoint)

    model.to(device)
    model.eval()
    print(f"物理制約統合済み PIGNNモデル のロードに成功しました！")
    print(f"ロード元: {save_path}")

    # 【テスト実行】戻り値の形式が変わっているか確認
    sample_data = next(iter(test_loader)).to(device)
    try:
        sample_data = preprocess_batch(sample_data, device)
        test_out = model(sample_data)
        if isinstance(test_out, tuple):
             print(f"物理バイアス出力確認: 正常 (戻り値は {len(test_out)} 要素)")
        else:
             print(f"警告: 戻り値が1つです。物理損失統合前のモデルかもしれません。")
    except Exception as e:
        print(f"動作確認中にエラー: {e}")
else:
    print(f"エラー: {save_path} が見つかりません。")

In [ ]:
# 最初のバッチを流してエラーが出ないかテスト
sample_data = next(iter(test_loader))

# 1. まずモデルをデバイスに送る（再確認）
model.to(device)
model.eval()

# 2. データをデバイスに送る (preprocess_batch 内で .to(device) されているか確認)
sample_data = preprocess_batch(sample_data, device)

with torch.no_grad():
    # 3. 推論を実行
    out = model(sample_data)
    print("推論テスト成功！ 出力形状:", out.shape) # [16, 2]

In [ ]:
def calculate_feature_importance_pignn(model, loader, device):
    model.eval()
    data_list = list(loader)

    # 特徴量リスト (6次元)
    node_features = ['pos_x', 'pos_y', 'vel_x', 'vel_y', 'dist_goal', 'dist_ball']
    all_names = node_features
    importances = []

    # ベースライン精度計測
    all_labels, all_preds = [], []
    with torch.no_grad():
        for data in data_list:
            data = preprocess_batch(data.clone())
            out = model(data)
            all_preds.extend(out.argmax(dim=1).cpu().numpy())
            all_labels.extend(data.y.view(-1).cpu().numpy())
    baseline_acc = (np.array(all_preds) == np.array(all_labels)).mean()

    for i in range(len(all_names)):
        preds = []
        with torch.no_grad():
            for data in data_list:
                batch_data = data.clone()
                batch_data = preprocess_batch(batch_data)
                # 該当特徴量を破壊
                batch_data.x[:, i] = 0
                out = model(batch_data)
                preds.extend(out.argmax(dim=1).cpu().numpy())

        acc = (np.array(preds) == np.array(all_labels)).mean()
        importances.append(max(0, baseline_acc - acc))

    return all_names, importances

In [ ]:
def calculate_feature_importance_pignn(model, loader, device):
    model.eval()
    data_list = list(loader)

    # 【重要修正】 特徴量リストを7次元に更新
    node_features = [
        'pos_x', 'pos_y',
        'vel_x', 'vel_y',
        'dist_goal', 'dist_ball',
        'team_id' # 7番目の新メンバー
    ]
    all_names = node_features
    importances = []

    # 1. ベースライン精度の計算
    all_labels, all_preds = [], []
    with torch.no_grad():
        for data in data_list:
            batch_data = preprocess_batch(data.clone(), device)
            out = model(batch_data)
            all_preds.extend(out.argmax(dim=1).cpu().numpy())
            all_labels.extend(batch_data.y.view(-1).cpu().numpy())

    baseline_acc = (np.array(all_preds) == np.array(all_labels)).mean()
    print(f"ベースライン精度 (Team-Aware v7): {baseline_acc:.4f}")

    # 2. 各特徴量を順番に破壊
    for i in range(len(all_names)):
        preds = []
        with torch.no_grad():
            for data in data_list:
                batch_data = data.clone()
                batch_data = preprocess_batch(batch_data, device)

                # i番目の特徴量を破壊
                batch_data.x[:, i] = 0

                out = model(batch_data)
                preds.extend(out.argmax(dim=1).cpu().numpy())

        acc = (np.array(preds) == np.array(all_labels)).mean()
        drop = baseline_acc - acc
        importances.append(max(0, drop))
        print(f"特徴量 '{all_names[i]}' 破壊時の精度低下: {drop:.4f}")

    return all_names, importances

# --- 実行 ---
feature_names, importance_values = calculate_feature_importance_pignn(model, test_loader, device)

アテンション係数の可視化

In [ ]:
#アテンション係数の可視化
import matplotlib.patches as patches

def visualize_pignn_final_v3(model, loader, sample_idx=0):
    model.eval()
    data = next(iter(loader)).to(device)

    # 推論
    input_data = data.clone()
    input_data = preprocess_batch(input_data, device)
    with torch.no_grad():
        out = model(input_data)
        probs = torch.exp(out)
        preds = out.argmax(dim=1)

    mask = (data.batch == sample_idx)
    pos = data.pos[mask].cpu().numpy()
    vel = data.vel[mask].cpu().numpy()

    # ノード数を確認（36個想定）
    num_nodes = pos.shape[0]

    fig, ax = plt.subplots(figsize=(12, 8))
    ax.set_facecolor('#f0f0f0')

    # ピッチ描画
    ax.add_patch(patches.Rectangle((-52.5, -34), 105, 68, fill=False, color='black', lw=2))
    ax.plot([0, 0], [-34, 34], color='black', alpha=0.3)

    for i in range(num_nodes):
        # --- 判定ロジックを「インデックス」に変更 ---
        if i == num_nodes - 1: # 最後のノードがボール
            color, marker, size, z = 'gold', '*', 450, 15
        elif i < 11: # 最初の11人が味方（仮定）
            color, marker, size, z = 'blue', 'o', 180, 10
        elif i < 22: # 次の11人が敵（仮定）
            color, marker, size, z = 'red', 'o', 180, 10
        else: # それ以外のダミー等
            color, marker, size, z = 'gray', 'o', 100, 5

        # 描画
        ax.scatter(pos[i, 0], pos[i, 1], c=color, marker=marker, s=size, edgecolors='black', zorder=z)
        # ベクトル
        ax.arrow(pos[i, 0], pos[i, 1], vel[i, 0]*1.5, vel[i, 1]*1.5,
                 head_width=1.0, head_length=1.2, fc=color, ec=color, alpha=0.4, zorder=z-1)

    plt.title(f"PIGNN Fixed Visualization\nPred: {preds[sample_idx]} | Prob: {probs[sample_idx, 1]:.2%}")
    plt.show()

# 実行
visualize_pignn_final_v3(model, test_loader, sample_idx=0)

学習曲線

In [ ]:
import matplotlib.pyplot as plt

# 学習時のループ内で train_acc も記録するように修正して実行したと仮定
# もし記録していなければ、このコードで現在の history からグラフを出します
plt.figure(figsize=(10, 6))
plt.plot(history['total_loss'], label='Train Loss (Total)')
# もし train_acc を取っていればここに追加
plt.plot(history['test_acc'], label='Test Accuracy', marker='o')

plt.title('Check for Overfitting: Loss vs Test Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Value')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

検証

ラベルをシャッフルした時の精度

In [ ]:
def shuffle_label_test(model, loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in loader:
            data = preprocess_batch(data, device)
            # ラベルをランダムにシャッフルする
            random_y = data.y[torch.randperm(data.y.size(0))]

            out = model(data)
            pred = out.argmax(dim=1)
            correct += (pred == random_y.view(-1)).sum().item()
            total += data.num_graphs

    print(f"ラベルシャッフル時の精度: {correct / total:.4f}")
    print(">> 0.5 (50%) 前後になれば、モデルは正しくラベルと特徴の関係を学んでいます。")

shuffle_label_test(model, test_loader, device)

物理損失（損失関数に付け加えた物理項）

In [ ]:
# 物理損失の平均値を算出
avg_phys = sum(history['physics_loss']) / len(history['physics_loss'])
print(f"平均物理損失: {avg_phys:.4f}")

if avg_phys < 25: # 18前後なら非常に優秀
    print(">> 物理的整合性は保たれています。AIは現実的な動きの範囲内で予測しています。")
else:
    print(">> 物理損失が高いです。AIが異常な速度を想定して予測している可能性があります。")

過学習を防ぐためにドロップアウト層を追加した修正版PIGNNモデル

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import global_mean_pool

class PIGNNClassifier_drop(nn.Module):
    def __init__(self, hidden_channels=64, dropout_rate=0.3):
        super(PIGNNClassifier_drop, self).__init__()
        self.dropout_rate = dropout_rate

        # 修正: 7次元入力 [x, y, vx, vy, d_goal, d_ball, team_id]
        self.conv1 = PIGNNLayer(7, hidden_channels)
        self.conv2 = PIGNNLayer(hidden_channels, hidden_channels)
        self.lin = nn.Linear(hidden_channels, 2)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        pos, vel = data.pos, data.vel

        # 1層目 + Dropout
        x = self.conv1(x, edge_index, pos, vel)
        x = F.elu(x)
        x = F.dropout(x, p=self.dropout_rate, training=self.training)

        # 2層目 + Dropout
        x = self.conv2(x, edge_index, pos, vel)
        x = F.elu(x)
        x = F.dropout(x, p=self.dropout_rate, training=self.training)

        x = global_mean_pool(x, batch)

        # 最終出力
        return F.log_softmax(self.lin(x), dim=1)

class PIGNNClassifier_v7_Final(nn.Module):
    def __init__(self, hidden_channels=64, dropout_rate=0.3):
        super(PIGNNClassifier_v7_Final, self).__init__()
        self.dropout_rate = dropout_rate

        # 修正: 7次元 [x, y, vx, vy, d_goal, d_ball, team_id]
        self.conv1 = PIGNNLayer(7, hidden_channels)
        self.conv2 = PIGNNLayer(hidden_channels, hidden_channels)
        self.lin = nn.Linear(hidden_channels, 2)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        pos, vel = data.pos, data.vel

        # 1層目 + Dropout
        x = F.elu(self.conv1(x, edge_index, pos, vel))
        x = F.dropout(x, p=self.dropout_rate, training=self.training)

        # 2層目 + Dropout
        x = F.elu(self.conv2(x, edge_index, pos, vel))
        x = F.dropout(x, p=self.dropout_rate, training=self.training)

        x = global_mean_pool(x, batch)
        return F.log_softmax(self.lin(x), dim=1)

In [ ]:
# 1. 新しいモデルをインスタンス化
model_dropout = PIGNNClassifier_drop(hidden_channels=64, dropout_rate=0.3).to(device)
optimizer_dropout = torch.optim.Adam(model_dropout.parameters(), lr=LR)

# 2. 履歴保存用（名前を分ける：history_dropout）
history_dropout = {
    'total_loss': [],
    'physics_loss': [],
    'test_acc': []
}

print(f"PIGNN学習開始（ドロップアウト版）")

for epoch in range(1, EPOCHS + 1):
    # model_dropout を使う
    avg_loss, avg_phys = train_pignn_epoch(model_dropout, train_loader, optimizer_dropout, ALPHA_P, device)
    acc = test_pignn(model_dropout, test_loader, device)

    history_dropout['total_loss'].append(avg_loss)
    history_dropout['physics_loss'].append(avg_phys)
    history_dropout['test_acc'].append(acc)

    if epoch % 5 == 0 or epoch == 1:
        print(f"Epoch {epoch:03d} | Loss: {avg_loss:.4f} | Phys_L: {avg_phys:.4f} | Acc: {acc:.4f}")

    # 保存ファイル名を変える（重要！）
    if epoch == 1 or acc > max(history_dropout['test_acc'][:-1]):
        torch.save(model_dropout.state_dict(), 'best_pignn_model_v6_dropout.pth')
        print(f" >> Model Saved (Best Dropout Acc: {acc:.4f})")

精度曲線

In [ ]:
#ドロップアウト層なしとありを重ねて描画

plt.figure(figsize=(10, 6))
plt.plot(history['test_acc'], label='Base Model (No Dropout)', alpha=0.6)
plt.plot(history_dropout['test_acc'], label='Improved Model (With Dropout)', linewidth=2)
plt.title('Effect of Dropout on Test Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

ベースラインより乱高下の幅が小さくなっている→改善ポイント

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# ==========================================
# 5. 最終評価とレポート (Dropout版・専用)
# ==========================================

# 1. ドロップアウト版のモデルをロード
# 学習時に保存した「dropout」という名前の方を読み込みます
model_drop_eval = PIGNNClassifier_drop(hidden_channels=64, dropout_rate=0.3).to(device)
model_drop_eval.load_state_dict(torch.load('best_pignn_model_v6_dropout.pth'))
model_drop_eval.eval()

all_preds = []
all_labels = []

# 2. テストデータで最終予測
with torch.no_grad():
    for data in test_loader:
        data = preprocess_batch(data, device)
        out = model_drop_eval(data)
        pred = out.argmax(dim=1)
        all_preds.extend(pred.cpu().numpy())
        all_labels.extend(data.y.view(-1).cpu().numpy())

# 3. レポート表示
print("\n" + "="*50)
print("       PIGNN 最終評価結果 (Dropout改良版)")
print("="*50)
print(classification_report(all_labels, all_preds, target_names=['Failure (0)', 'Success (1)']))

# 4. 混同行列の描画
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Oranges', # 区別するために色をオレンジ系に
            xticklabels=['Fail', 'Success'], yticklabels=['Fail', 'Success'])
plt.title('Confusion Matrix (PIGNN with Dropout)')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

精度は下がったが、モデルは改善していると言える。

検証

In [ ]:
def shuffle_label_test_dropout_ver(model_to_test, loader, device):
    model_to_test.eval()
    correct = 0
    total = 0

    # 判定用の閾値（不均衡データの場合、多数派の割合に引っ張られるため）
    # 今回のテストデータ Failure:239, Success:90 なので、ランダムなら
    # (239/329)^2 + (90/329)^2 ≒ 0.6 くらいになるのが統計学的な「勘」の限界です。

    with torch.no_grad():
        for data in loader:
            data = preprocess_batch(data, device)

            # ラベルをランダムにシャッフル
            random_y = data.y[torch.randperm(data.y.size(0))]

            out = model_to_test(data)
            pred = out.argmax(dim=1)

            correct += (pred == random_y.view(-1)).sum().item()
            total += data.num_graphs

    shuffle_acc = correct / total
    print(f"ラベルシャッフル時の精度: {shuffle_acc:.4f}")

    if shuffle_acc < 0.58: # 0.62から下がっていれば改善
        print(">> 合格：暗記（過学習）が抑制され、特徴量とラベルの真の相関を学んでいます。")
    else:
        print(">> 警告：依然としてデータの偏り（初期配置など）を強く覚えすぎている可能性があります。")

# 実行（ドロップアウト版のモデルを指定）
shuffle_label_test_dropout_ver(model_drop_eval, test_loader, device)

物理損失

In [ ]:
# ==========================================
# 物理的整合性の最終確認 (Dropout版)
# ==========================================

# history_dropout の中身を使って計算します
if 'physics_loss' in history_dropout and len(history_dropout['physics_loss']) > 0:
    avg_phys = sum(history_dropout['physics_loss']) / len(history_dropout['physics_loss'])
    print(f"ドロップアウト版 平均物理損失: {avg_phys:.4f}")

    # 判定
    if avg_phys < 25:
        print(">> 合格：物理的整合性は保たれています。")
        print(">> ドロップアウトを導入しても、AIは現実的な物理法則（速度ベクトル）を無視していません。")
    else:
        print(">> 警告：物理損失が増大しています。")
        print(">> 汎化性能を優先するあまり、物理レイヤーの制約が弱まっている可能性があります。")
else:
    print(">> エラー：history_dropout に physics_loss が記録されていません。")

In [ ]:
import torch
import matplotlib.pyplot as plt
from torch_geometric.explain import Explainer, GNNExplainer
from torch_geometric.data import Data

# --- 修正の核心：重要度を抽出する1行 ---
# node_mask は (ノード数, 特徴量数) の形状なので、全ノードで平均して特徴量ごとの重要度にする
importances = explanation.node_mask.mean(dim=0)

# --- 以降、あなたの可視化コードへ続く ---
if torch.is_tensor(importances):
    importances = importances.cpu().numpy()

# 1. バラバラの引数を「dataオブジェクト」に梱包してモデルに渡すラッパー
class ExplainerWrapper(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, x, edge_index, batch=None, **kwargs):
        # GNNExplainerから届く各テンソルを、モデルが期待する Data オブジェクトに擬似再現
        # pos と vel は x の中に入っている、あるいは別途渡されることを想定
        # あなたのモデル定義に合わせて x から pos, vel を切り出す、
        # または data.pos, data.vel としてアクセスできるようにします。

        # 仮に x の 0-1列目が pos, 2-3列目が vel だと想定される場合：
        # (モデルの入力に合わせて調整してください。x自体に全て含まれているならそのままでも可)
        tmp_data = Data(x=x, edge_index=edge_index, batch=batch)

        # もしモデルが data.pos や data.vel を直接参照しているなら、ここで代入
        # x の構成が [特徴量...] で、pos/velが別管理なら以下のように復元
        tmp_data.pos = x[:, :2]  # 例: 最初の2列が座標
        tmp_data.vel = x[:, 2:4] # 例: 次の2列が速度

        return self.model(tmp_data)

# ラップしたモデルを作成
wrapped_model = ExplainerWrapper(model_drop_eval)

# 2. Explainerの設定
model_config = {
    'mode': 'multiclass_classification',
    'task_level': 'graph',
    'return_type': 'log_probs',
}

explainer = Explainer(
    model=wrapped_model,
    algorithm=GNNExplainer(epochs=200),
    explanation_type='model',
    node_mask_type='attributes',
    edge_mask_type='object',
    model_config=model_config,
)

# --- データの準備 ---
test_batch = next(iter(test_loader))
test_batch = preprocess_batch(test_batch, device)
data_list = test_batch.to_data_list()
data_single = data_list[0]

# 3. 重要度の算出
explanation = explainer(
    x=data_single.x,
    edge_index=data_single.edge_index,
    batch=torch.zeros(data_single.x.size(0), dtype=torch.long).to(device)
)

print("重要度の算出に成功しました！")

# --- 修正版：重要度の可視化コード ---

# ラベルを7次元用に更新
labels = ['x', 'y', 'vx', 'vy', 'dist_goal', 'dist_ball', 'team_id']

# importances が numpy 形式であることを確認
if torch.is_tensor(importances):
    importances = importances.cpu().numpy()

plt.figure(figsize=(10, 6))
# ここで次元数を自動で合わせます
plt.bar(labels, importances, color='teal')

plt.title('GNNExplainer: Feature Importance (PIGNN v7)')
plt.ylabel('Importance Score')
plt.grid(axis='y', linestyle='--', alpha=0.7)

# 値を棒の上に表示
for i, v in enumerate(importances):
    plt.text(i, v + 0.01, f"{v:.3f}", ha='center')

plt.show()

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# 1. 解析の設定
num_samples = 100
all_node_importances = []

print(f"{num_samples}シーンの解析を開始します。勾配計算を有効にして最適化を行うため、少し時間がかかります...")

# モデルを評価モードにしつつ、GNNExplainer内部の学習は許可する
model_drop_eval.eval()

# 進捗管理用のカウンタ
count = 0

for data in tqdm(test_loader):
    if count >= num_samples:
        break

    data = preprocess_batch(data, device)
    data_list = data.to_data_list()

    for data_single in data_list:
        if count >= num_samples:
            break

        # --- 修正ポイント：with torch.no_grad() を削除 ---
        # GNNExplainerは内部でロスを計算し .backward() を呼ぶため勾配が必要
        explanation = explainer(
            x=data_single.x,
            edge_index=data_single.edge_index,
            batch=torch.zeros(data_single.x.size(0), dtype=torch.long).to(device)
        )

        node_importance = explanation.node_mask.abs().mean(dim=0).cpu().numpy()
        all_node_importances.append(node_importance)
        count += 1

# 3. 平均と標準誤差の計算
avg_importance = np.mean(all_node_importances, axis=0)
std_importance = np.std(all_node_importances, axis=0) / np.sqrt(len(all_node_importances))

# --- 修正版：ラベルとグラフ描画 ---

# 1. ラベルを現在の7次元仕様に完全に合わせる
# [x, y, vx, vy, dist_goal, dist_ball, team_id]
labels = ['PosX', 'PosY', 'VelX', 'VelY', 'DistGoal', 'DistBall', 'TeamID']

plt.figure(figsize=(12, 7))

# 2. データの数とラベルの数が一致しているか確認して描画
# avg_importance と std_importance が 7要素であることを前提とします
bars = plt.bar(labels, avg_importance, yerr=std_importance,
               color='teal', edgecolor='navy', capsize=5, alpha=0.8)

plt.title('GNNExplainer: Mean Feature Importance over 100 Scenes (v7)', fontsize=14)
plt.ylabel('Importance Score', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.5)

# 棒の上に数値を表示
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.01,
             f'{height:.3f}', ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
# ==========================================
# 成功と失敗の比較解析 (PIGNN v7対応版)
# ==========================================
import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# 1. データの仕分け用リスト
success_importances = []
failure_importances = []

num_samples = 150  # 統計的安定性のために150シーンを推奨
count = 0

print(f"{num_samples}シーンを成功/失敗別に解析します...")

# モデルを評価モードに
model_drop_eval.eval()

# tqdmで進捗を表示しながらループ
for data in tqdm(test_loader):
    if count >= num_samples:
        break

    # バッチをデバイスに送り、個別データに分解
    data = preprocess_batch(data, device)
    data_list = data.to_data_list()

    for data_single in data_list:
        if count >= num_samples:
            break

        # GNNExplainerで重要度算出
        explanation = explainer(
            x=data_single.x,
            edge_index=data_single.edge_index,
            # batchテンソルもデバイスに合わせる
            batch=torch.zeros(data_single.x.size(0), dtype=torch.long).to(device)
        )

        # ノード特徴量の重要度（絶対値の平均）を取得
        node_importance = explanation.node_mask.abs().mean(dim=0).cpu().numpy()

        # 正解ラベル(y)に基づいて仕分け
        if data_single.y.item() == 1:
            success_importances.append(node_importance)
        else:
            failure_importances.append(node_importance)

        count += 1

# 2. 【重要】ラベルを7次元（v7仕様）に更新
labels = ['PosX', 'PosY', 'VelX', 'VelY', 'DistGoal', 'DistBall', 'TeamID']

# 各グループの平均を算出
# ここで avg_success の shape は (7,) になります
avg_success = np.mean(success_importances, axis=0)
avg_failure = np.mean(failure_importances, axis=0)

# 3. 比較グラフの描画
x = np.arange(len(labels)) # 0から6までのインデックス
width = 0.35

fig, ax = plt.subplots(figsize=(14, 8))

# 棒グラフの描画
rects1 = ax.bar(x - width/2, avg_success, width, label='Success (1)', color='forestgreen', alpha=0.8)
rects2 = ax.bar(x + width/2, avg_failure, width, label='Failure (0)', color='crimson', alpha=0.8)

# グラフの装飾
ax.set_ylabel('Mean Importance Score', fontsize=12)
ax.set_title('Feature Importance Comparison: Success vs Failure (PIGNN v7)', fontsize=14)
ax.set_xticks(x)
ax.set_xticklabels(labels, fontsize=11)
ax.legend(fontsize=12)
ax.grid(axis='y', linestyle='--', alpha=0.3)

# 数値ラベルを表示する補助関数
def autolabel(rects):
    for rect in rects:
        height = rect.get_height()
        ax.annotate(f'{height:.3f}',
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3), # 3pt上に表示
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=9)

autolabel(rects1)
autolabel(rects2)

plt.tight_layout()
plt.show()

# 4. 数値の要約表示
print("\n--- 解析結果の要約 ---")
for i, label in enumerate(labels):
    diff = avg_success[i] - avg_failure[i]
    trend = "↑ Successで重視" if diff > 0 else "↓ Failureで重視"
    print(f"[{label}] Success: {avg_success[i]:.4f} | Failure: {avg_failure[i]:.4f} | {trend}")

バイアスの可視化

In [ ]:
# 描画の前にこれを入れてください
sample = next(iter(test_loader))
# 最初の1バッチ分のチームIDの中身をすべて表示
print("--- TeamID Raw Data Check ---")
print(sample.x[:, 6])
print("-----------------------------")

# もしここで 0.0 しか出てこないなら、データの作り直しが必要

In [ ]:
def analyze_physics_bias(model, data, tau=1.5):
    """
    モデル内の物理バイアスを解析する関数（名前を修正）
    """
    model.eval()
    with torch.no_grad():
        pos = data.pos
        vel = data.vel
        edge_index = data.edge_index

        # 未来位置の予測
        pos_pred = pos + vel * tau

        # エッジごとの未来距離と物理バイアス
        row, col = edge_index
        dist_future = torch.norm(pos_pred[row] - pos_pred[col], dim=-1)
        physics_bias = torch.exp(-dist_future / 1.0)#5.0から1.0に修正

        # 最大バイアスの取得
        top_idx = torch.argmax(physics_bias)
        top_pair = (edge_index[0, top_idx].item(), edge_index[1, top_idx].item())

    # 戻り値のキーを 'max_bias' に修正してエラーを解消
    return {
        "top_pair": top_pair,
        "max_bias": physics_bias[top_idx].item(), # ここを bias_value から max_bias へ修正
        "pos_pred": pos_pred
    }

In [ ]:
def visualize_pignn_v7_absolute_colors(model, loader, sample_idx=0, tau=1.5):
    model.eval()
    # データを1つ取得
    batch = next(iter(loader)).to(device)
    input_data = preprocess_batch(batch.clone(), device)

    with torch.no_grad():
        out = model(input_data)
        probs = torch.exp(out)
        preds = out.argmax(dim=1)

    # 描画対象のインデックスを抽出
    mask = (batch.batch == sample_idx)
    pos = batch.pos[mask].cpu().numpy()
    vel = batch.vel[mask].cpu().numpy()

    # 【最重要】team_id (index 6) を直接取得して中身を確認
    team_ids = input_data.x[mask, 6].cpu().numpy()

    fig, ax = plt.subplots(figsize=(12, 8))
    # ピッチ背景
    ax.set_facecolor('#f0f0f0')
    ax.add_patch(plt.Rectangle((-52.5, -34), 105, 68, fill=False, color='black', lw=2))
    ax.plot([0, 0], [-34, 34], color='black', alpha=0.3)

    for i in range(len(pos)):
        t_val = team_ids[i]

        # --- 判定ロジックを「範囲」にして誤差を許容 ---
        if t_val > 1.5:          # Ball (2.0)
            color, marker, size, z = '#FFD700', '*', 500, 15 # Gold
            lbl = "Ball"
        elif t_val > 0.5:        # Defender (1.0)
            color, marker, size, z = '#EE3333', 'o', 250, 10 # Red
            lbl = "Defender (Away)"
        else:                    # Attacker (0.0)
            color, marker, size, z = '#3366FF', 'o', 250, 10 # Blue
            lbl = "Attacker (Home)"

        # 描画
        ax.scatter(pos[i, 0], pos[i, 1], c=color, marker=marker, s=size,
                   edgecolors='black', linewidths=1.2, zorder=z, label=lbl)

        # 速度ベクトル
        ax.arrow(pos[i, 0], pos[i, 1], vel[i, 0]*tau, vel[i, 1]*tau,
                 head_width=0.8, head_length=1.0, fc=color, ec=color,
                 alpha=0.3, zorder=z-1)

    # 物理バイアスの描画（緑のX印と点線）
    res = analyze_physics_bias(model, batch.to_data_list()[sample_idx], tau=tau)
    p1, p2 = res["top_pair"] # タプルなのでそのまま受け取るだけでOK
    p1, p2 = int(p1), int(p2) # 念のため整数型に変換
    ax.plot([pos[p1,0], pos[p2,0]], [pos[p1,1], pos[p2,1]], 'green', linestyle='--', lw=2, alpha=0.6)
    ax.scatter(res["pos_pred"][p1,0].cpu(), res["pos_pred"][p1,1].cpu(),
               color='green', marker='X', s=350, edgecolors='white', label='Conflict Point', zorder=20)

    # テキスト情報
    res_str = "SUCCESS" if preds[sample_idx] == 1 else "FAILURE"
    plt.title(f"PIGNN v7 Tactical Analysis: {res_str}\n"
              f"AI Prediction Prob: {probs[sample_idx, 1]:.2%} | Max Physics Bias: {res['max_bias']:.3f}",
              fontsize=14, fontweight='bold')

    # 凡例の重複を削除
    handles, labels = ax.get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    ax.legend(by_label.values(), by_label.keys(), loc='upper right', frameon=True, shadow=True)

    plt.tight_layout()
    plt.show()

# 実行
visualize_pignn_v7_absolute_colors(model, test_loader, sample_idx=0)

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch_geometric.loader import DataLoader

# ==========================================
# 1. テストセット全体から最小バイアスを探索
# ==========================================
model.eval()
min_bias = float('inf')
low_bias_data = None
low_bias_idx = -1

print("テストセットから最も物理的に安定したシーンを探索中...")

with torch.no_grad():
    # test_set は Data オブジェクトのリストであることを前提としています
    for i, data in enumerate(test_set):
        # データのデバイス移動
        data_to_device = data.to(device)

        # 解析関数を呼び出し（tau=1.5秒後の未来を予測）
        res = analyze_physics_bias(model, data_to_device, tau=1.5)

        current_max_bias = res['max_bias']

        # 最小値を更新
        if current_max_bias < min_bias:
            min_bias = current_max_bias
            low_bias_idx = i
            low_bias_data = data_to_device

print(f"✅ 探索完了")
print(f"発見された最小バイアス: {min_bias:.4f}")
print(f"該当データのインデックス: {low_bias_idx}")

# ==========================================
# 2. 修正版：IndexError回避用可視化呼び出し
# ==========================================

def visualize_specific_scene(model, data_list, target_idx, tau=1.5):
    """
    特定のインデックスのデータだけを抽出し、
    バッチとして可視化関数に渡すことで IndexError を防ぐ
    """
    # ターゲットのデータ1枚だけを含むリストを作成
    single_data_list = [data_list[target_idx]]

    # バッチサイズ1の専用ローダーを作成
    single_loader = DataLoader(single_data_list, batch_size=1)

    # 既存の可視化関数を呼び出し
    # バッチ内のインデックスは必ず 0 になる
    visualize_pignn_v7_absolute_colors(model, single_loader, sample_idx=0, tau=tau)

# 実行：物理的に最も「綺麗」なシーンを描画
visualize_specific_scene(model, test_set, low_bias_idx, tau=1.5)

In [ ]:
def find_most_intense_duel(model, data_list, tau=1.5):
    model.eval()
    max_duel_bias = -1.0
    best_idx = -1

    print("攻守が最も激しく『ぶつかる』シーンを探索中...")

    with torch.no_grad():
        for i, data in enumerate(data_list):
            data_to_device = data.to(device)
            # 全ペアのバイアス詳細を取得
            res = analyze_physics_bias(model, data_to_device, tau=tau)

            p1, p2 = res["top_pair"]
            # チームIDを取得 (index 6)
            t1 = data.x[p1, 6].item()
            t2 = data.x[p2, 6].item()

            # 異なるチーム同士（0.0:Home vs 1.0:Away）の衝突のみをターゲットにする
            # 1.0 - 0.0 = 1.0 の絶対値で判定
            if abs(t1 - t2) == 1.0:
                if res['max_bias'] > max_duel_bias:
                    max_duel_bias = res['max_bias']
                    best_idx = i

    print(f"発見！ 最大攻守衝突バイアス: {max_duel_bias:.4f} (Index: {best_idx})")
    return best_idx

# 1. 激しい競り合いシーンを特定
duel_idx = find_most_intense_duel(model, test_set)

# 2. 可視化
if duel_idx != -1:
    visualize_specific_scene(model, test_set, duel_idx, tau=1.5)
else:
    print("条件に合うシーンが見つかりませんでした。")